In [1]:
from pymongo import MongoClient
from datetime import datetime as dt

from src.shared import Guild, Channel, ChannelType, Message, User
from src.server import db, asset_generator

In [2]:
# dangerous!!!
db.guilds.drop()
db.channels.drop()
db.messages.drop()
db.users.drop()

In [3]:
u = asset_generator.generate_user("Test User")

In [4]:
guilds = [asset_generator.generate_guild(f"Test guild {i}", u.id) for i in range(5)]

In [5]:
for guild in guilds:
    for i in range(5):
        asset_generator.generate_channel(f"test channel {i}", ChannelType.TEXT, guild.id)

In [6]:
for channel_db in db.channels.find():
    channel: Channel = Channel.from_db_dict(channel_db)
    buffer: list[dict] = []
    for i in range(150):
        id = asset_generator.get_id()
        buffer.insert(0, Message(id, channel.id, f"test message {i} in channel {channel.name} in guild {channel.guild_id}", attachment_id=0, author=u, timestamp=id >> asset_generator.TAG_LENGTH).to_db_dict())
    db.messages.insert_many(buffer)
db.messages.create_index("channel_id")

'channel_id_1'

In [7]:
db.get_messages(channel.id, 0, 10)

[Message(id=7190392915103428, channel_id=7190392910905374, content='test message 149 in channel test channel 4 in guild 7190392910905349', attachment_id=0, author=User(id=7190392910905344, username='Test User'), timestamp=1714323262),
 Message(id=7190392915103427, channel_id=7190392910905374, content='test message 148 in channel test channel 4 in guild 7190392910905349', attachment_id=0, author=User(id=7190392910905344, username='Test User'), timestamp=1714323262),
 Message(id=7190392915103426, channel_id=7190392910905374, content='test message 147 in channel test channel 4 in guild 7190392910905349', attachment_id=0, author=User(id=7190392910905344, username='Test User'), timestamp=1714323262),
 Message(id=7190392915103425, channel_id=7190392910905374, content='test message 146 in channel test channel 4 in guild 7190392910905349', attachment_id=0, author=User(id=7190392910905344, username='Test User'), timestamp=1714323262),
 Message(id=7190392915103424, channel_id=7190392910905374, c